# 07 Regression Algorithm dengan KNN (K Nearest Neighbours)


Dalam **Regression K Nearest Neighbours (KNN)** digunakan untuk mengestimasi variable berkelanjutan atau kontinu. Salah satu algoritma tersebut menggunakan rata-rata `KNN`, yang dibobotkan oleh kebalikan jarak dari mereka. Algoritma ini bekerja sebagai berikut :

`1`. Hitung jarak Euclidean atau Mahalanobis dari contoh query ke contoh berlabel.

`2`. Mengurutkan contoh berlabel dengan menambah jarak.

`3`. Menemukan jumlah `KNN` yang optimal secara heuristik, berdasarkan RMSE. Ini dilakukan dengan menggunakan validasi silang.

`4`. Menghitung rata-rata tertimbang jarak terbalik dengan k-nearest multivariate neighbors.

- KNN adalah model machine learning yang dapat digunakan untuk melakukan prediksi berdasarkan kedekatan karakteristik dengan sejumlah tetangga terdekat.
- Prediksi yang dilakukan dapat diterapkan baik pada classification maupun regression tasks.

Referensi: [https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)

## Sample Dataset

Pada tahap `Sample Dataset` terlebih dahulu kita untuk melakukan `import pandas as pd` dengan mempersiapkan `sample_data` yang akan di olah. Sample Dataset yang digunakan adalah `sensus` sebagai variabel dengan format pandas `DataFrame` berisikan :
- `tinggi` dan `jk` sebagai **Features**
- `berat` (jenis kelamin) sebagai **Target**

In [1]:
import pandas as pd

sensus = {'tinggi': [158, 170, 183, 191, 155, 163, 180, 158, 170], 
          'jk': ['pria', 'pria', 'pria', 'pria', 'wanita', 'wanita', 'wanita', 'wanita', 'wanita'],
          'berat': [64, 86, 84, 80, 49, 59, 67, 54, 67]}

sensus_df = pd.DataFrame(sensus)
sensus_df

,tinggi,jk,berat
0,158,pria,64
1,170,pria,86
2,183,pria,84
3,191,pria,80
4,155,wanita,49
5,163,wanita,59
6,180,wanita,67
7,158,wanita,54
8,170,wanita,67


## Regression dengan KNN


Pada tahap ini kita akan coba untuk melakukan estimasi prediksi terhadap nilai `berat`.

### Features & Target

Sebelum melakukan **Training Model** kita akan kelompokkan terlebih dahulu sekumpulan nilai `Features` dan `Target` dari Dataset yang dimiliki. Pada tahap ini dikonversikan ke dalam `numpy` (np.array) array dengan melakukan `import numpy as np`.
- `X_train` sebagai Features untuk training set yang  berisikan `tinggi` dan `jk`.
- `y_train` sebagai Features untuk training set yang  berisikan `berat`.

In [2]:
import numpy as np

X_train = np.array(sensus_df[['tinggi', 'jk']])
y_train = np.array(sensus_df['berat'])

print(f'X_train:\n{X_train}\n')
print(f'y_train: {y_train}')

X_train:
[[158 'pria']
 [170 'pria']
 [183 'pria']
 [191 'pria']
 [155 'wanita']
 [163 'wanita']
 [180 'wanita']
 [158 'wanita']
 [170 'wanita']]

y_train: [64 86 84 80 49 59 67 54 67]


- Untuk tahap selanjutnya kita perlu memastikan bahwa tipe data **Features** adalah numerik agar dapat dihitung jarak antar data pointnya. Pada tahap berikutnya kita akan konversikan nilai data **pria** dan **wanita** menjadi nilai `0` dan `1`

### Preprocess Dataset: Konversi Label menjadi Numerik Biner

Melakukan Konversi `pria` dan `wanita` menjadi Numerik Biner pertama-tama kita menggunakan proses `transpose` terlebih dahulu dengan parameter `X_train` yang ditampung ke dalam variable `X_train_transposed`.

Proses `transpose` ini pada umumnya akan mengubah posisi `baris` menjadi `kolom` dan sebaliknya.
- `baris` sebagai intens.
- `kolom` sebagai features.

In [3]:
X_train_transposed = np.transpose(X_train)

print(f'X_train:\n{X_train}\n')
print(f'X_train_transposed:\n{X_train_transposed}')

X_train:
[[158 'pria']
 [170 'pria']
 [183 'pria']
 [191 'pria']
 [155 'wanita']
 [163 'wanita']
 [180 'wanita']
 [158 'wanita']
 [170 'wanita']]

X_train_transposed:
[[158 170 183 191 155 163 180 158 170]
 ['pria' 'pria' 'pria' 'pria' 'wanita' 'wanita' 'wanita' 'wanita'
  'wanita']]


Sebelum melakukan konversi nilai `String` ke `Numerik` dengan **Binarization** terlebih dahulu kita melakukan import `from sklearn.preprocessing import LabelBinarizer`.
- `lb = LabelBinarizer()` variabel `lb` sebagai `LabelBinarizer()`
- `jk_binarised = lb.fit_transform(X_train_transposed[1])` sebagai transformasi data yang terdapat di **X_train_transposed** dengan indeks ke-1 karena itu adalah jenis kelamin `jk` yang ditampung ke dalam variabel `jk_binarised`.
- `print(f'jk: {X_train_transposed[1]}\n')` untuk menampilkan data sebelum di konversikan.
- `print(f'jk_binarised:\n{jk_binarised}')` untuk menampilkan konversi Binarize.

Nilai yang dihasilkan adalah sebagai berikut :
- Nilai **0** sebagai `pria`
- Nilai **1** sebagai `wanita`

In [4]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
jk_binarised = lb.fit_transform(X_train_transposed[1])

print(f'jk: {X_train_transposed[1]}\n')
print(f'jk_binarised:\n{jk_binarised}')

jk: ['pria' 'pria' 'pria' 'pria' 'wanita' 'wanita' 'wanita' 'wanita' 'wanita']

jk_binarised:
[[0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]]


Setelah hasil data berhasil ditransformasi ke dalam `Binarize` datanya berubah menjadi Array 2 dimensi. Oleh karenanya Kita dapat mengubah kedalam `Array 1 Dimenensi` dengan **flattern()** dengan menggunakan script dibawah ini.

In [5]:
jk_binarised = jk_binarised.flatten()
jk_binarised

array([0, 0, 0, 0, 1, 1, 1, 1, 1])

Tahapan berikutnya adalah nilai yang sudah dikonversikan sebelumnya akan kita tampung lagi ke dalam **`X_train_transposed`** karena nilai tersebut masih berada dalam variabel **`jk_binarised`** dengan indeks ke-1 karena berkolerasi dengan jenis kelamin (jk).

- Selanjutnya nilai **`X_train_transposed`** akan kita transpose balik agar yang tadinya `baris` menjadi `kolom` dan sebaliknya yang akan ditampung ke dalam variabel `X_train` dengan menggunakan script dibawah ini.

In [6]:
X_train_transposed[1] = jk_binarised
X_train = X_train_transposed.transpose()

print(f'X_train_transposed:\n{X_train_transposed}\n')
print(f'X_train:\n{X_train}')

X_train_transposed:
[[158 170 183 191 155 163 180 158 170]
 [0 0 0 0 1 1 1 1 1]]

X_train:
[[158 0]
 [170 0]
 [183 0]
 [191 0]
 [155 1]
 [163 1]
 [180 1]
 [158 1]
 [170 1]]


### Training KNN Regression Model

Setelah **Training Set** sudah disiapkan selanjutnya kita akan melakukan **Training Model** dengan `KNN`. Regression taks juga sering kali dikenal dengan istilah **Regressor**.

Pertama-tama kita melakukan import dengan `from sklearn.neighbors import KNeighborsRegressor` dengan keterangan sebagai berikut :
- `K` memiliki nilai `3`
- `model = KNeighborsRegressor(n_neighbors=K)` sebagai objek model `KNeighborsRegressor` dengan parameter `n_neighbors=K` yang ditampung ke dalam variabel `mdoel`.
- `model.fit(X_train, y_train)` sebagai training model dengan `X_train` dan `y_train` sebagai parameternya dengan `model.fit`

In [7]:
from sklearn.neighbors import KNeighborsRegressor

K = 3
model = KNeighborsRegressor(n_neighbors=K)
model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

### Prediksi Berat Badan

Tahapan ini kita akan melakukan prediksi berat badan berdasarkan `tinggi` dan `jenis kelamin (jk)`, pada kasus ini kita siapkan data baru dengan `X_new` sebagai variabel dan `np.array([[155, 1]])` memiliki parameter :

- Tinggi `155`
- Jenis Kelamin (jk) `1` sebagai wanita.

In [8]:
X_new = np.array([[155, 1]])
X_new

array([[155,   1]])

Tahap berikutnya kita akan memprediksi berat badan dengan memanfaatkan `KNN Regressor` yang sudah ditraining sebelumnya dengan menggunakan `model.predict` dengan parameter `X_new` yang ditampung ke dalam variabel `y_pred`.

In [9]:
y_pred = model.predict(X_new)
y_pred

array([55.66666667])

Hasil diatas menunjukkan bahwa data dengan tinggi badan `155` yang berjenis kelamin (jk) wanita yaitu `1` memiliki berat badan `55.66666667`.

### Evaluasi KNN Regression Model

Pada tahap ini kita akan mulai mempelajari beberapa matriks yang bisa kita gunakan untuk mengukur peforma dari `Regression Taks`, sebelum melakukannya kita persiapkan terlebih dahulu `Testing Set` sebgai berikut :

- `X_test = np.array([[168, 0], [180, 0], [160, 1], [169, 1]])` sebagai nilai **Features** yang ditampung ke dalam variabel `X_test`.
- `y_test = np.array([65, 96, 52, 67])` sebagai nilai **Target** yang ditampung ke dalam variabel `y_test`.

In [10]:
X_test = np.array([[168, 0], [180, 0], [160, 1], [169, 1]])
y_test = np.array([65, 96, 52, 67])

print(f'X_test:\n{X_test}\n')
print(f'y_test: {y_test}')

X_test:
[[168   0]
 [180   0]
 [160   1]
 [169   1]]

y_test: [65 96 52 67]


Tahap berikutnya kita akan lakukan predikis terhadap `Testing Set` ini dengan memanfaatkan `KNN Regressor` yang sudah ditraining sebelumnya yang menggunakan `model.predict` dengan parameter `X_test` yang ditampung ke dalam variabel `y_pred`.

In [11]:
y_pred = model.predict(X_test)
y_pred

array([70.66666667, 79.        , 59.        , 70.66666667])

Hasil diatas menunjukkan bahwa data sebagai berikut :

- Tinggi badan `168` yang berjenis kelamin (jk) pria yaitu `0` memiliki berat badan `70.66666667`. Sedangkan data yang diharapkan adalah `65`.
- Untuk data terkahir dengan Tinggi badan `169` yang berjenis kelamin (jk) wanita yaitu `1` memiliki berat badan `70.66666667`. Sedangkan data yang diharapkan adalah `67`.

#### Coefficient of Determination atau $R^2$

**`Coefficient of Determination`**, denoted R2 or r2 and pronounced `"R squared"`, is the proportion of the variance in the dependent variable that is predictable from the independent variable(s).

It is a statistic used in the context of statistical models whose main purpose is either the prediction of future outcomes or the testing of hypotheses, on the basis of other related information. It provides a measure of how well observed outcomes are replicated by the model, based on the proportion of total variation of outcomes explained by the model.

Referensi: [https://en.wikipedia.org/wiki/Coefficient_of_determination](https://en.wikipedia.org/wiki/Coefficient_of_determination)


Sebelum melakukan `R2` terlebih dahulu kita melakukan import `from sklearn.metrics import r2_score` dengan keterangan sebagai berikut :
- `r_squared = r2_score(y_test, y_pred)` dengan r2_score memiliki 2 parameter yaitu `y_test` dan `y_pred` yang ditampung ke dalam variabel `r_squared`.
- `print(f'R-squared: {r_squared}')` untuk menampilkan hasil `r_squared`. Jika hasil semakin mendekati nilai `1` akan lebih baik daripada hasil mendekati `0` seperti pada penjelasan sebelumnya.

In [12]:
from sklearn.metrics import r2_score

r_squared = r2_score(y_test, y_pred)

print(f'R-squared: {r_squared}')

R-squared: 0.6290565226735438


#### Mean Absolute Error (MAE) atau Mean Absolute Deviation (MAD)

$MAE$ is the average of the absolute values of the errors of the predictions.

**`Mean Absolute Error (MAE)`** is a measure of errors between paired observations expressing the same phenomenon. Examples of Y versus X include comparisons of predicted versus observed, subsequent time versus initial time, and one technique of measurement versus an alternative technique of measurement. MAE dikalkulasikan sebagai berikut :

$MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$

- $y_i$

Information of the Calculate MAE :
- $y_i$ merepresentasikan setiap nilai Target pada Testing Set.
- $\hat{y}_i$ adalah nilai prediksi yang dihasilkan oleh kita.
- Nilai **n** merupakan nilai jumlah data point.

Untuk menghindari nilai negatif kita gunakan `absolute` function.

Referensi: [https://en.wikipedia.org/wiki/Mean_absolute_error](https://en.wikipedia.org/wiki/Mean_absolute_error)

Selanjutnya untuk menggunakan `Mean Absolute Error (MAE)` telebih dahulu kita melakukan import `from sklearn.metrics import mean_absolute_error` dengan keterangan sebagai berikut :

- `MAE = mean_absolute_error(y_test, y_pred)` dengan mean_absolute_error memiliki 2 parameter yaitu `y_test` dan `y_pred`.
- `print(f'MAE: {MAE}')` untuk menampilkan hasil dari MAE.

In [13]:
from sklearn.metrics import mean_absolute_error

MAE = mean_absolute_error(y_test, y_pred)

print(f'MAE: {MAE}')

MAE: 8.333333333333336


#### Mean Squared Error (MSE) atau Mean Squared Deviation (MSD)

$MSE$ is the average of the squares of the errors of the predictions.

**Mean Squared Error (MSE) or Mean Squared Deviation (MSD)** of an estimator (of a procedure for estimating an unobserved quantity) measures the average of the squares of the errors—that is, the average squared difference between the estimated values and the actual value. MSE is a risk function, corresponding to the expected value of the squared error loss. The fact that MSE is almost always strictly positive (and not zero) is because of randomness or because the estimator does not account for information that could produce a more accurate estimate. MSE atau MSD dikalkulasikan sebagai berikut :

$MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$


Referensi: [https://en.wikipedia.org/wiki/Mean_squared_error](https://en.wikipedia.org/wiki/Mean_squared_error)

Selanjutnya untuk menggunakan `Mean Squared Error (MSE) or Mean Squared Deviation (MSD)` telebih dahulu kita melakukan import `from sklearn.metrics import mean_squared_error` dengan keterangan sebagai berikut :

- `MSE = mean_squared_error(y_test, y_pred)` dengan mean_squared_error memiliki 2 parameter yaitu `y_test` dan `y_pred`.
- `print(f'MSE: {MSE}')` untuk menampilkan hasil dari MSE.


Untuk menghindari kemunculan nilai negatif $(y_i - \hat{y}_i)^2$ hasil selisih harus dipangkatkan dengan 2

In [14]:
from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(y_test, y_pred)

print(f'MSE: {MSE}')

MSE: 95.8888888888889


Jika hasil dari `Mean Squared Error (MSE) or Mean Squared Deviation (MSD)` lebih kecil akan didefiniskan lebih baik dikarenakan ini untuk menghitung dari `Mean Sqaured Error`.

### Permasalahan Scaling pada Features

**Feature Scaling** adalah suatu cara untuk membuat numerical data pada dataset memiliki rentang nilai (scale) yang sama. Tidak ada lagi satu variabel data yang mendominasi variabel data lainnya.

Referensi : https://medium.com/machine-learning-id/melakukan-feature-scaling-pada-dataset-229531bb08de#:~:text=Feature%20Scaling%20adalah%20suatu%20cara,yang%20mendominasi%20variabel%20data%20lainnya.

Untuk melakukan Scaling pada Features terlebih dahulu kita melakukan import dengan `from scipy.spatial.distance import euclidean` pada kasus ini **`tinggi dalam milimeter`** dengan keterangan sebagai berikut :

- `X_train = np.array([[1700, 0], [1600, 1]])` dengan np.array memiliki 2 parameter yaitu `[1700, 0]` dan `[1600, 1]` yang ditampung ke dalam variabel `X_train`.
- `X_new = np.array([[1640, 0]])` dengan np.array memiliki parameter yaitu `[1640, 0]` yang ditampung ke dalam variabel `X_new`.
- `[euclidean(X_new[0], d) for d in X_train]` sebagai proses pengukuran untuk mengetahui apakah hasilnya akan berdampak pada eksistensi **`euclidean`** seperti script dibawah ini.

In [15]:
from scipy.spatial.distance import euclidean

# tinggi dalam milimeter
X_train = np.array([[1700, 0], [1600, 1]])
X_new = np.array([[1640, 0]])

[euclidean(X_new[0], d) for d in X_train]

[60.0, 40.01249804748511]

Untuk melakukan Scaling pada Features terlebih dahulu kita melakukan import dengan `from scipy.spatial.distance import euclidean` pada kasus ini **`tinggi dalam meter`** dengan keterangan sebagai berikut :

- `X_train = np.array([[1.7, 0], [1.6, 1]])` dengan np.array memiliki 2 parameter yaitu `[1.7, 0]` dan `[1.6, 1]])` yang ditampung ke dalam variabel `X_train`.
- `X_new = np.array([[1.64, 0]])` dengan np.array memiliki parameter yaitu `[1640, 0]` yang ditampung ke dalam variabel `X_new`.
- `[euclidean(X_new[0], d) for d in X_train]` untuk mengetahui apakah hasilnya akan berdampak pada eksistensi **`euclidean`** seperti script dibawah ini.

In [16]:
# tinggi dalam meter
X_train = np.array([[1.7, 0], [1.6, 1]])
X_new = np.array([[1.64, 0]])

[euclidean(X_new[0], d) for d in X_train]

[0.06000000000000005, 1.0007996802557444]

### Menerapkan Standard Scaler (Standard Score atau Z-Score)

Standardize features by removing the mean and scaling to unit variance.

**Standard Score** is the number of standard deviations by which the value of a raw score (i.e., an observed value or data point) is above or below the mean value of what is being observed or measured. Raw scores above the mean have positive standard scores, while those below the mean have negative standard scores.

It is calculated by subtracting the population mean from an individual raw score and then dividing the difference by the population standard deviation. This process of converting a raw score into a standard score is called standardizing or normalizing (however, "normalizing" can refer to many types of ratios; see normalization for more). Z-Score dikalkulasikan sebagai berikut :


$z = \frac{x - \bar{x}}{S}$

information of the Calculate :
- $x$ sebagai nilai Features.
- $\bar{x}$ sebagai nilai rata-rata dari nilai **Z**.
- $(S)$ sebagai nilai standard deviation dari sekumpulan nilai Features.


Referensi: [https://en.wikipedia.org/wiki/Standard_score](https://en.wikipedia.org/wiki/Standard_score)

Untuk melakukan **Standard Scaler (Standard Score atau Z-Score)** terlebih dahulu kita melakukan import dengan `from sklearn.preprocessing import StandardScaler` dengan keterangan sebagai berikut :

- `ss = StandardScaler()` dengan fungsi StandardScaler() yang ditampung ke dalam variabel `ss` seperti scrript dibawah ini.

In [17]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

### Standard Scaler dengan Tinggi dalam Milimeter

Untuk melakukan Scaling pada Features terlebih dahulu kita melakukan import dengan `from scipy.spatial.distance import euclidean` pada kasus ini **`tinggi dalam milimeter`** dengan keterangan sebagai berikut :

**X_train_scaled :**
- `X_train = np.array([[1700, 0], [1600, 1]])` dengan np.array memiliki 2 parameter yaitu `[1700, 0]` dan `[1600, 1]` yang ditampung ke dalam variabel `X_train`.
- `X_train_scaled = ss.fit_transform(X_train)` dengan ss.fit_transform memiliki parameter yaitu `X_train` yang ditampung ke dalam variabel `X_train_scaled`.

**X_new_scaled :**
- `X_new = np.array([[1640, 0]])` dengan np.array memiliki parameter yaitu `[1640, 0]` yang ditampung ke dalam variabel `X_new`.
- `X_new_scaled = ss.transform(X_new)` dengan ss.fit_transform memiliki parameter yaitu `X_new` yang ditampung ke dalam variabel `X_new_scaled`.


- `jarak = [euclidean(X_new_scaled[0], d) for d in X_train_scaled]` sebagai proses pengukuran yang ditampung ke dalam variabel `jarak` untuk mengetahui apakah hasilnya akan berdampak pada eksistensi **`euclidean`** seperti script dibawah ini.

In [18]:
# tinggi dalam milimeter
X_train = np.array([[1700, 0], [1600, 1]])
X_train_scaled = ss.fit_transform(X_train)
print(f'X_train_scaled:\n{X_train_scaled}\n')

X_new = np.array([[1640, 0]])
X_new_scaled = ss.transform(X_new)
print(f'X_new_scaled: {X_new_scaled}\n')

jarak = [euclidean(X_new_scaled[0], d) for d in X_train_scaled]
print(f'jarak: {jarak}')

X_train_scaled:
[[ 1. -1.]
 [-1.  1.]]

X_new_scaled: [[-0.2 -1. ]]

jarak: [1.2, 2.154065922853802]


### Standard Scaler dengan Tinggi dalam Meter

Untuk melakukan Scaling pada Features terlebih dahulu kita melakukan import dengan `from scipy.spatial.distance import euclidean` pada kasus ini **`tinggi dalam milimeter`** dengan keterangan sebagai berikut :

**X_train_scaled :**
- `X_train = np.array([[1.7, 0], [1.6, 1]])` dengan np.array memiliki 2 parameter yaitu `[1.7, 0]` dan `[1.6, 1]` yang ditampung ke dalam variabel `X_train`.
- `X_train_scaled = ss.fit_transform(X_train)` dengan ss.fit_transform memiliki parameter yaitu `X_train` yang ditampung ke dalam variabel `X_train_scaled`.

**X_new_scaled :**
- `X_new = np.array([[1.64, 0]])` dengan np.array memiliki parameter yaitu `[1.64, 0]` yang ditampung ke dalam variabel `X_new`.
- `X_new_scaled = ss.transform(X_new)` dengan ss.fit_transform memiliki parameter yaitu `X_new` yang ditampung ke dalam variabel `X_new_scaled`.


- `jarak = [euclidean(X_new_scaled[0], d) for d in X_train_scaled]` sebagai proses pengukuran yang ditampung ke dalam variabel `jarak` untuk mengetahui apakah hasilnya akan berdampak pada eksistensi **`euclidean`** seperti script dibawah ini.

In [19]:
# tinggi dalam meter
X_train = np.array([[1.7, 0], [1.6, 1]])
X_train_scaled = ss.fit_transform(X_train)
print(f'X_train_scaled:\n{X_train_scaled}\n')

X_new = np.array([[1.64, 0]])
X_new_scaled = ss.transform(X_new)
print(f'X_new_scaled: {X_new_scaled}\n')

jarak = [euclidean(X_new_scaled[0], d) for d in X_train_scaled]
print(f'jarak: {jarak}')

X_train_scaled:
[[ 1. -1.]
 [-1.  1.]]

X_new_scaled: [[-0.2 -1. ]]

jarak: [1.2000000000000026, 2.1540659228538006]


### Menerapkan Features Scaling pada KNN

#### Dataset

Sebelumnya kita siapkan terlebih dahulu sebagai berikut :
**Training Set**
- `X_train` sebagai variabel dengan sekumpulan nilai Features
- `y_train` sebagai variabel dengan sekumpulan nilai Target

**Test Set** sebagai keperluan evaluasi yaitu `X_test` dan `y_test`.

In [20]:
# Training Set
X_train = np.array([[158, 0], [170, 0], [183, 0], [191, 0], [155, 1], [163, 1],
                    [180, 1], [158, 1], [170, 1]])

y_train = np.array([64, 86, 84, 80, 49, 59, 67, 54, 67])

# Test Set
X_test = np.array([[168, 0], [180, 0], [160, 1], [169, 1]])
y_test = np.array([65, 96, 52, 67])

#### Features Scaling (Standard Scaler)

- `X_train_scaled = ss.fit_transform(X_train)` memiliki ss.fit_transform sebagai fit_transform yang memiliki parameter `X_train` yang ditampung ke dalam variabel `X_train_scaled`.
- `X_test_scaled = ss.transform(X_test)` memiliki ss.transform sebagai transform yang memiliki parameter `X_test` yang ditampung ke dalam variabel `X_test_scaled`.

In [21]:
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

print(f'X_train_scaled:\n{X_train_scaled}\n')
print(f'X_test_scaled:\n{X_test_scaled}\n')

X_train_scaled:
[[-0.9908706  -1.11803399]
 [ 0.01869567 -1.11803399]
 [ 1.11239246 -1.11803399]
 [ 1.78543664 -1.11803399]
 [-1.24326216  0.89442719]
 [-0.57021798  0.89442719]
 [ 0.86000089  0.89442719]
 [-0.9908706   0.89442719]
 [ 0.01869567  0.89442719]]

X_test_scaled:
[[-0.14956537 -1.11803399]
 [ 0.86000089 -1.11803399]
 [-0.82260955  0.89442719]
 [-0.06543485  0.89442719]]



#### Training & Evaluasi Model

Pada tahap `Training dan Evaluasi Model` terlebih dahulu kita melakukan model `model.fit(X_train_scaled, y_train)` memiliki 2 parameter yaitu `X_train_scaled` dan `y_train` dengan keterangan sebagai berikut :

- `y_pred = model.predict(X_test_scaled)` melakukan prediksi dengan model.predict yang memiliki parameter yaitu `X_test_scaled`.

- `MAE = mean_absolute_error(y_test, y_pred)` melakukan prediksi dengan mean_absolute_error yang memiliki 2 parameter yaitu `y_test` dan `y_pred` yang ditampung ke dalam variabel `MAE`.

- `MSE = mean_squared_error(y_test, y_pred)` melakukan prediksi dengan mean_squared_error yang memiliki 2 parameter yaitu `y_test` dan `y_pred` yang ditampung ke dalam variabel `MSE`.

Seperti script dibawah ini.

In [22]:
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

MAE = mean_absolute_error(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)

print(f'MAE: {MAE}')
print(f'MSE: {MSE}')

MAE: 7.583333333333336
MSE: 85.13888888888893
